# Pearson Dfs EGFR phospho sites vs all phospho sites

This notebooks makes df for each cancer type that compares an EGFR phospho site against all phospho sites. (returns all)

In [5]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import re
import sys 
import statsmodels.stats.multitest


import cptac
import cptac.utils as u

In [6]:

'''
@Param df: Dataframe. Contains numeric values (such as proteomics) for linear regression
@Param label_column: String. Name of column that will be your x axis and will be compared to all values in df unless otherwise specified. 
@Param alpha: significant level
@Param comparison_columns: columns that will be looped through and used as y axis for linear regression. 
All other columns beside label column unless specified here. 
@Param correction_method: String. Specifies method of adjustment for multiple testing. See -
https://www.statsmodels.org/stable/generated/statsmodels.stats.multitest.multipletests.html
    - for documentation and available methods.

This function will return a data frame will all significant linear regressions. The data frame includes the comparison, slope, R-squared, and P-value. 
'''
def wrap_lin_regression(df,label_column, alpha=.05,comparison_columns=None,correction_method='bonferroni',return_all = True):
    

    
    '''If no comparison columns specified, use all columns except the specified labed column'''
    if not comparison_columns:
        comparison_columns = list(df.columns)
        comparison_columns.remove(label_column)
    '''Store comparisons,p-values, correlation in their own array'''
    comparisons = []
    pvals = []
    correlation=[]
    
    
    '''Format results in a pandas dataframe'''
    newdf = pd.DataFrame(columns=['Comparison','Correlation','P_value'])
    for inter_gene in comparison_columns:
        #create subset df with interacting gene/ gene (otherwise drop NaN drops everything)
        df_subset = df[[label_column,inter_gene]]
        #do a linear regression to see if it's a meaningful association
        #dropna will remove rows with nan
        df_subset = df_subset.dropna(axis=0, how="any")
        count_row = df_subset.shape[0]
        if count_row > 20:
            x1 = df_subset[[label_column]].values
            y1 = df_subset[[inter_gene]].values
            x1 = x1[:,0]
            y1 = y1[:,0]

            #slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(x1,y1)
            corr, pval = scipy.stats.pearsonr(x1,y1)

            comparisons.append(inter_gene)
            pvals.append(pval)
            correlation.append(corr)
            #slope_val.append(slope)
        
    '''Correct for multiple testing to determine if each comparison meets the new cutoff'''
    results = statsmodels.stats.multitest.multipletests(pvals=pvals, alpha=alpha, method=correction_method)
    reject = results[0]
        
    if return_all:
        for i in range(0,len(comparisons)):
            newdf = newdf.append({'Comparison': comparisons[i],"Correlation": correlation[i],'P_value': pvals[i]}, ignore_index=True)
        
    '''Else only add significant comparisons'''
    if (return_all == False):
            for i in range(0, len(reject)):
                if reject[i]:
                    newdf = newdf.append({'Comparison': comparisons[i],"Slope": slope_val[i], 'R_squared': r_squared[i], 'P_value': pvals[i]}, ignore_index=True)
                    
    '''Sort dataframe by ascending p-value'''
    newdf = newdf.sort_values(by='P_value', ascending=True)
    '''If results df is not empty, return it, else return None'''
    return newdf



       
       

In [ ]:
def get_phospho_sites(df):
    phospho_sites = df.columns.values.tolist()
    phospho_sites.remove('EGFR_Mutation_')
    phospho_sites.remove('EGFR_Location_')
    phospho_sites.remove('EGFR_Mutation_Status_')
    return phospho_sites 


In [4]:
#load cptac data 
brain = cptac.Gbm()
kidney = cptac.Ccrcc()
Ovar = cptac.Ovarian()
colon = cptac.Colon()
brca = cptac.Brca()
luad = cptac.Luad()
Hnscc = cptac.Hnscc()
Lscc = cptac.Lscc()

cptac warning: The GBM dataset is under publication embargo until March 01, 2021. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 2)


cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 7)


# Step 1 Make data frame 

For each cancer, get phosphoproteomic and mutation data. Drop extra multi-index levels. Merge levels to have column names include gene and phospho site 

# Step 2 Get Phospho Sites

For each cancer, get phospho sites by using get_phospho_sites function. Function extracts column names and deletes unnecessary columns. It returns list of phospho sites. 

# Step 3 Run Pearson Correlation Function

Run correlation function and compare EGFR phospho site to all phospho sites. Save df as csv file. 

# GBM

In [7]:
df1 = brain.join_omics_to_mutations(omics_df_name="phosphoproteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(2)
df1.columns = df1.columns.droplevel(2)
df1.columns =df1.columns.map('_'.join)
phosphoprot = df1.loc[df1['Sample_Status_'] == "Tumor"]

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 93 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [8]:


phospho_sites = phosphoprot.columns.values.tolist()

phospho_sites.remove('EGFR_Mutation_')

phospho_sites.remove('EGFR_Location_')

phospho_sites.remove('EGFR_Mutation_Status_')


In [ ]:
df =(wrap_lin_regression(phosphoprot,"EGFR_phosphoproteomics_Y1197",comparison_columns = phospho_sites, return_all = True))
df['Cancer Type']='GBM'
#df.to_csv("GBM_EGFR_Y1197_phospho.csv")

/Users/Lindsey/anaconda3/lib/python3.7/site-packages/scipy/stats/stats.py:3399: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:153: RuntimeWarning: invalid value encountered in less_equal
  reject = pvals <= alphacBonf
/Users/Lindsey/anaconda3/lib/python3.7/site-packages/statsmodels/stats/multitest.py:257: RuntimeWarning: invalid value encountered in greater
  pvals_corrected[pvals_corrected>1] = 1


In [7]:

df

,Comparison,Correlation,P_value,Cancer Type
14365,EGFR_phosphoproteomics_Y1197,1.000000,0.000000e+00,GBM
14364,EGFR_phosphoproteomics_Y1172,0.899603,8.871131e-34,GBM
14348,EGFR_phosphoproteomics_S1166Y1172,0.881333,1.002277e-30,GBM
14362,EGFR_phosphoproteomics_Y1092,0.908060,3.304647e-23,GBM
14347,EGFR_phosphoproteomics_S1166,0.739790,5.458615e-17,GBM
...,...,...,...,...
5451,BAZ1B_phosphoproteomics_S708,-0.000009,9.999604e-01,GBM
33328,PALMD_phosphoproteomics_S486,0.000008,9.999697e-01,GBM
815,ADD3_phosphoproteomics_S677S679S681,-0.000002,9.999883e-01,GBM
22015,ITPRID2_phosphoproteomics_T867,-0.000002,9.999904e-01,GBM


In [8]:
#cptac.download(dataset='ccrcc')
kidney = cptac.Ccrcc()


In [9]:
df1 = kidney.join_omics_to_mutations(omics_df_name="phosphoproteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(2)
df1.columns = df1.columns.droplevel(2)
df1.columns =df1.columns.map('_'.join)


df2 = df1.loc[df1['Sample_Status_'] == "Tumor"]

df3 = df2.loc[:,~df1.columns.duplicated()]
phospho_sites = df2.columns.values.tolist()

phospho_sites.remove('EGFR_Mutation_')

phospho_sites.remove('EGFR_Location_')

phospho_sites.remove('EGFR_Mutation_Status_')

phospho_sites.remove('Sample_Status_')

df3.filter(like='EGFR_phosphoproteomics_Y', axis=1)

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 193 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


,EGFR_phosphoproteomics_Y1016,EGFR_phosphoproteomics_Y1039,EGFR_phosphoproteomics_Y1119,EGFR_phosphoproteomics_Y1144,EGFR_phosphoproteomics_Y963
Patient_ID,,,,,
C3L-00004,NaN,0.285696,NaN,0.999000,-0.013055
C3L-00010,NaN,0.601337,0.412977,0.700833,NaN
C3L-00011,NaN,-0.239413,0.271045,-0.430652,NaN
C3L-00026,NaN,0.444422,1.011546,0.465533,0.460270
C3L-00079,NaN,NaN,-0.000500,NaN,0.040108
...,...,...,...,...,...
C3N-01646,NaN,-0.160831,NaN,-0.404793,NaN
C3N-01648,NaN,-0.413072,NaN,-0.650702,NaN
C3N-01649,NaN,0.353399,-0.059095,0.369224,-0.250412


In [10]:
df =(wrap_lin_regression(df3,"EGFR_phosphoproteomics_Y1144",comparison_columns = phospho_sites, return_all = True))
df['Cancer Type']='Kidney'
df.to_csv("Kidney_EGFR_Y1144_phospho.csv")

# Ovarian 

In [11]:
#cptac.download(dataset='ovarian')
Ovar = cptac.Ovarian()

In [12]:
df1 = Ovar.join_omics_to_mutations(omics_df_name="phosphoproteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(2)
df1.columns = df1.columns.droplevel(2)
df1.columns =df1.columns.map('_'.join)


df2 = df1.loc[df1['Sample_Status_'] == "Tumor"]

df3 = df2.loc[:,~df1.columns.duplicated()]
phospho_sites = df3.columns.values.tolist()

phospho_sites.remove('EGFR_Mutation_')

phospho_sites.remove('EGFR_Location_')

phospho_sites.remove('EGFR_Mutation_Status_')


phospho_sites.remove('Sample_Status_')
df3

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 102 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


,AAAS_phosphoproteomics_S495,AAGAB_phosphoproteomics_S311,AAK1_phosphoproteomics_S18,AAK1_phosphoproteomics_S20,AAK1_phosphoproteomics_S21,AAK1_phosphoproteomics_S624,AAK1_phosphoproteomics_S637,AAK1_phosphoproteomics_S642,AAK1_phosphoproteomics_S668S678,AAK1_phosphoproteomics_S668T674,...,ZZZ3_phosphoproteomics_S135,ZZZ3_phosphoproteomics_S314,ZZZ3_phosphoproteomics_S391,ZZZ3_phosphoproteomics_S89,ZZZ3_phosphoproteomics_S90,ZZZ3_phosphoproteomics_S91,EGFR_Mutation_,EGFR_Location_,EGFR_Mutation_Status_,Sample_Status_
Patient_ID,,,,,,,,,,,,,,,,,,,,,
01OV007,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.829,NaN,NaN,...,NaN,-2.533,NaN,NaN,NaN,-1.320,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
01OV017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.095,NaN,NaN,...,NaN,-3.276,NaN,NaN,NaN,-0.264,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
01OV018,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.748,NaN,NaN,...,NaN,NaN,-0.847,NaN,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
01OV023,-1.137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.185,NaN,...,NaN,NaN,-0.455,NaN,NaN,-1.129,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
01OV026,-0.987,NaN,NaN,NaN,-2.269,-3.363,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,-1.301,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26OV002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.921,...,NaN,NaN,NaN,NaN,NaN,-1.232,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
26OV008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1.796,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.134,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
26OV009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2.795,NaN,NaN,...,NaN,NaN,NaN,NaN,-2.503,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor


In [13]:
df =(wrap_lin_regression(df3,"EGFR_phosphoproteomics_Y1172",comparison_columns = phospho_sites, return_all = True))
df['Cancer Type']='Ovar'
df.to_csv("Ovar_EGFR_Y1172_phospho.csv")
df
#df3.filter(like='EGFR_phosphoproteomics_Y', axis=1)


,Comparison,Correlation,P_value,Cancer Type
854,EGFR_phosphoproteomics_Y1172,1.000000,0.000000e+00,Ovar
1689,MARK2_phosphoproteomics_S40,0.805187,4.084338e-07,Ovar
853,EGFR_phosphoproteomics_S1166,0.789336,5.966751e-07,Ovar
253,ARHGEF5_phosphoproteomics_S606,0.722735,9.442209e-07,Ovar
1379,KDM3B_phosphoproteomics_S779,0.721595,1.000808e-06,Ovar
...,...,...,...,...
2466,RGS14_phosphoproteomics_S203,-0.000776,9.972646e-01,Ovar
3141,TNS1_phosphoproteomics_S1327,-0.000529,9.981828e-01,Ovar
3458,ZNF687_phosphoproteomics_S253,0.000242,9.989014e-01,Ovar
2977,TANC2_phosphoproteomics_S1740,-0.000299,9.989753e-01,Ovar


# Colon

In [14]:
#cptac.download(dataset='colon')
colon = cptac.Colon()

In [15]:
df1 = colon.join_omics_to_mutations(omics_df_name="phosphoproteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(2)
#df1.columns = df1.columns.droplevel(2)
df1.columns =df1.columns.map('_'.join)


df2 = df1.loc[df1['Sample_Status_'] == "Tumor"]

df3 = df2.loc[:,~df1.columns.duplicated()]
phospho_sites = df3.columns.values.tolist()

phospho_sites.remove('EGFR_Mutation_')

phospho_sites.remove('EGFR_Location_')

phospho_sites.remove('EGFR_Mutation_Status_')


phospho_sites.remove('Sample_Status_')


cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 204 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)
cptac warning: phosphoproteomics data was not found for the following samples, so phosphoproteomics data columns were filled with NaN for these samples: 01CO001, 05CO005, 05CO007, 05CO014, 05CO055, 11CO059, 11CO060, 14CO002, 16CO012, 24CO005 (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


In [16]:

df3.filter(like='EGFR_phosphoproteomics_Y', axis=1)

,EGFR_phosphoproteomics_Y1092,EGFR_phosphoproteomics_Y1172,EGFR_phosphoproteomics_Y1197
Patient_ID,,,
01CO001,NaN,NaN,NaN
01CO005,NaN,NaN,-1.002
01CO006,0.178,-0.421,NaN
01CO008,-0.784,NaN,NaN
01CO013,NaN,NaN,NaN
...,...,...,...
21CO007,-0.396,NaN,NaN
22CO004,NaN,NaN,NaN
22CO006,-0.400,NaN,NaN


In [17]:
df =(wrap_lin_regression(df3,"EGFR_phosphoproteomics_Y1092",comparison_columns = phospho_sites, return_all = True))
df['Cancer Type']='Colon'
df.to_csv("colon_EGFR_Y1092_phospho.csv")
df


,Comparison,Correlation,P_value,Cancer Type
2396,EGFR_phosphoproteomics_Y1092,1.000000,0.000000,Colon
9747,WNK2_phosphoproteomics_S1889,0.737281,0.000017,Colon
4514,MAP1B_phosphoproteomics_S1400,0.627090,0.000056,Colon
5180,NCAM1_phosphoproteomics_S784,0.623998,0.000062,Colon
4874,MID1_phosphoproteomics_S98,0.697698,0.000106,Colon
...,...,...,...,...
9911,ZC3HC1_phosphoproteomics_S354,0.000099,0.999563,Colon
797,ARVCF_phosphoproteomics_S348,-0.000081,0.999685,Colon
8619,SYNPO_phosphoproteomics_S882,-0.000074,0.999747,Colon
8129,SRRM1_phosphoproteomics_S463,0.000024,0.999890,Colon


# Brca 

In [18]:
#cptac.download(dataset='brca')
brca = cptac.Brca()

In [19]:
df1 = brca.join_omics_to_mutations(omics_df_name="phosphoproteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(2)
df1.columns = df1.columns.droplevel(2)
df1.columns =df1.columns.map('_'.join)


df2 = df1.loc[df1['Sample_Status_'] == "Tumor"]

df3 = df2.loc[:,~df1.columns.duplicated()]
phospho_sites = df3.columns.values.tolist()

phospho_sites.remove('EGFR_Mutation_')

phospho_sites.remove('EGFR_Location_')

phospho_sites.remove('EGFR_Mutation_Status_')


phospho_sites.remove('Sample_Status_')

df3

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 118 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


,A2M_phosphoproteomics_S710,AAAS_phosphoproteomics_S495,AAAS_phosphoproteomics_S541,AAED1_phosphoproteomics_S12,AAGAB_phosphoproteomics_S310S311,AAGAB_phosphoproteomics_S311,AAK1_phosphoproteomics_S14,AAK1_phosphoproteomics_S18,AAK1_phosphoproteomics_S21,AAK1_phosphoproteomics_S618T620S623,...,ZZZ3_phosphoproteomics_S391,ZZZ3_phosphoproteomics_S397,ZZZ3_phosphoproteomics_S397T428N429,ZZZ3_phosphoproteomics_S606,ZZZ3_phosphoproteomics_S82,ZZZ3_phosphoproteomics_S89,EGFR_Mutation_,EGFR_Location_,EGFR_Mutation_Status_,Sample_Status_
Patient_ID,,,,,,,,,,,,,,,,,,,,,
CPT000814,NaN,1.9431,NaN,NaN,0.0127,-0.4495,NaN,-1.1852,-0.8333,0.0863,...,-8.8556,NaN,-0.8493,-0.1744,-0.0273,-6.8916,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
CPT001846,NaN,0.2274,NaN,0.5472,0.6895,-0.7680,NaN,0.6895,0.3903,NaN,...,-4.0222,NaN,1.1806,-1.2700,-0.8983,-3.6228,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
X01BR001,NaN,-2.2853,-0.8967,4.1225,0.4842,-1.2458,-0.0310,0.8805,0.7448,1.0649,...,-1.6299,-0.2017,-0.3953,-2.1328,-0.8815,-2.1191,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
X01BR008,NaN,1.3714,NaN,-0.0636,-0.6224,-1.2732,-1.1351,-0.8652,0.0031,0.2173,...,1.3333,0.7840,0.1078,0.4824,NaN,-0.9748,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
X01BR009,NaN,0.2682,NaN,NaN,-0.6207,-2.3561,-0.3078,0.3785,0.6990,-0.1082,...,0.5267,-0.6661,0.5539,-0.3728,NaN,-0.8354,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
X21BR001,NaN,0.9424,NaN,0.4838,0.0477,-0.3365,-0.4697,-0.4213,0.1117,-0.5217,...,0.9078,0.7330,1.5429,0.6015,NaN,-2.2315,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
X21BR002,NaN,0.2955,NaN,-0.0053,0.1169,0.5654,-0.7218,-0.2368,0.2974,-0.0491,...,-0.4611,-0.3007,0.4122,-0.6325,NaN,-0.4046,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
X21BR010,-0.7266,-0.1201,-0.1477,1.8818,0.8338,0.2280,-0.0877,-0.4698,-0.2266,-0.0214,...,-2.2380,0.9095,0.3844,0.1160,-3.4806,0.0170,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor


In [20]:

df3.filter(like='EGFR_phosphoproteomics', axis=1)
#no EGFR phospho Y data 

,EGFR_phosphoproteomics_S1039,EGFR_phosphoproteomics_S1064,EGFR_phosphoproteomics_S1166,EGFR_phosphoproteomics_S991,EGFR_phosphoproteomics_T1041S1042,EGFR_phosphoproteomics_T693
Patient_ID,,,,,,
CPT000814,-0.2928,0.1774,-1.9944,0.1950,0.1806,-0.3343
CPT001846,1.9046,NaN,5.2070,0.4808,NaN,3.1401
X01BR001,NaN,-0.3739,3.2933,NaN,NaN,1.3256
X01BR008,-0.5764,NaN,-0.8240,NaN,-0.1732,-0.5732
X01BR009,0.4722,NaN,1.6650,NaN,-0.6993,0.0248
...,...,...,...,...,...,...
X21BR001,-0.2240,NaN,-2.5568,NaN,-0.3555,-1.7590
X21BR002,0.1734,NaN,-0.3882,NaN,-1.0938,-0.9023
X21BR010,-0.0334,NaN,0.5478,-1.3929,-1.0773,-0.9196


# LUAD

In [21]:
#cptac.download(dataset='luad')
luad = cptac.Luad()

cptac warning: The LUAD dataset is under publication embargo until July 01, 2020. CPTAC is a community resource project and data are made available rapidly after generation for community research use. The embargo allows exploring and utilizing the data, but analysis may not be published until after the embargo date. Please see https://proteomics.cancer.gov/data-portal/about/data-use-agreement or enter cptac.embargo() to open the webpage for more details. (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 2)


In [22]:
df1 = luad.join_omics_to_mutations(omics_df_name="phosphoproteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(2)
df1.columns = df1.columns.droplevel(2)
df1.columns =df1.columns.map('_'.join)


df2 = df1.loc[df1['Sample_Status_'] == "Tumor"]

df3 = df2.loc[:,~df1.columns.duplicated()]
phospho_sites = df3.columns.values.tolist()

phospho_sites.remove('EGFR_Mutation_')

phospho_sites.remove('EGFR_Location_')

phospho_sites.remove('EGFR_Mutation_Status_')


phospho_sites.remove('Sample_Status_')

df3

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 171 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


,AAAS_phosphoproteomics_S495,AAAS_phosphoproteomics_S541,AAED1_phosphoproteomics_S12,AAGAB_phosphoproteomics_S310S311,AAGAB_phosphoproteomics_S311,AAK1_phosphoproteomics_S20,AAK1_phosphoproteomics_S618T620S624,AAK1_phosphoproteomics_S624,AAK1_phosphoproteomics_S678,AAK1_phosphoproteomics_S682,...,ZZEF1_phosphoproteomics_T66,ZZZ3_phosphoproteomics_S113,ZZZ3_phosphoproteomics_S391,ZZZ3_phosphoproteomics_S397N429,ZZZ3_phosphoproteomics_S606,ZZZ3_phosphoproteomics_S89,EGFR_Mutation_,EGFR_Location_,EGFR_Mutation_Status_,Sample_Status_
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00001,-0.7127,0.2683,0.9238,0.1711,NaN,1.0165,NaN,-0.2064,-0.0278,NaN,...,NaN,-1.0721,-0.3262,-0.1611,NaN,0.6164,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00009,0.6344,NaN,1.3945,-2.3271,-0.4007,1.5299,NaN,-0.1799,1.5237,0.1158,...,0.3657,-0.1195,0.9010,0.2720,0.7177,-0.9776,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00080,1.0390,NaN,-0.2910,NaN,0.2226,0.5386,NaN,-1.0926,NaN,-1.4284,...,-0.6087,-0.3008,0.7575,NaN,0.1502,0.1946,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00083,-0.4906,NaN,0.8506,NaN,0.3430,0.2118,NaN,2.5134,1.7586,-1.1010,...,NaN,-0.2718,0.7937,0.3386,0.4415,-1.7661,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00093,1.0305,-0.5968,0.3249,1.0423,-0.2167,1.1706,1.0851,NaN,0.4152,0.6599,...,NaN,0.0422,0.4389,NaN,0.1586,0.0303,"[Missense_Mutation, Missense_Mutation]","[p.L858R, p.A871G]",Multiple_mutation,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-02729,0.0377,NaN,-0.4162,NaN,-1.3180,1.2554,-0.2403,-2.7458,-1.0661,-0.9181,...,NaN,-0.2763,-1.3520,NaN,NaN,-1.1201,[Missense_Mutation],[p.L858R],Single_mutation,Tumor
X11LU013,0.4598,NaN,-1.3742,-0.4182,1.1134,-0.3382,NaN,NaN,1.5914,0.5378,...,NaN,-1.2122,-0.3557,NaN,-0.5020,-2.0238,[In_Frame_Del],[p.E746_S752delinsA],Single_mutation,Tumor
X11LU016,0.8232,0.5637,0.3794,NaN,1.0695,-0.5345,-0.8673,NaN,1.1221,-0.4329,...,0.1143,-0.1998,0.1237,NaN,-0.5815,-1.0779,"[Missense_Mutation, Missense_Mutation]","[p.G719C, p.S768I]",Multiple_mutation,Tumor


In [23]:

df3.filter(like='EGFR_phosphoproteomics_Y', axis=1)

,EGFR_phosphoproteomics_Y1092,EGFR_phosphoproteomics_Y1172,EGFR_phosphoproteomics_Y1197
Patient_ID,,,
C3L-00001,NaN,5.9688,8.5140
C3L-00009,-2.0647,-1.0692,-1.3295
C3L-00080,NaN,NaN,-2.1724
C3L-00083,NaN,-2.7310,-2.2124
C3L-00093,NaN,-0.7893,0.7454
...,...,...,...
C3N-02729,NaN,2.8111,1.8433
X11LU013,NaN,1.0763,1.3651
X11LU016,NaN,-0.3352,-1.1907


In [24]:
df =(wrap_lin_regression(df3,"EGFR_phosphoproteomics_Y1197",comparison_columns = phospho_sites, return_all = True))
df['Cancer Type']='Luad'
df.to_csv("Luad_EGFR_Y1197_phospho.csv")
df


,Comparison,Correlation,P_value,Cancer Type
10047,EGFR_phosphoproteomics_Y1197,1.000000e+00,0.000000e+00,Luad
10046,EGFR_phosphoproteomics_Y1172,8.817770e-01,1.885106e-31,Luad
10038,EGFR_phosphoproteomics_S1064,7.289959e-01,2.552188e-17,Luad
10040,EGFR_phosphoproteomics_S1166,6.916211e-01,4.397719e-15,Luad
10044,EGFR_phosphoproteomics_T693,6.863959e-01,8.495371e-15,Luad
...,...,...,...,...
4378,BRMS1L_phosphoproteomics_S174,4.159763e-05,9.997572e-01,Luad
12232,FLNA_phosphoproteomics_S1081,4.075534e-05,9.997577e-01,Luad
16132,KANSL1_phosphoproteomics_S991,3.754599e-05,9.997914e-01,Luad
1647,ANK3_phosphoproteomics_S1824,1.305538e-05,9.998987e-01,Luad


# HNSCC

In [33]:
#cptac.download(dataset='hnscc')
Hnscc = cptac.Hnscc()

In [26]:
df1 = Hnscc.join_omics_to_mutations(omics_df_name="phosphoproteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(2)
df1.columns = df1.columns.droplevel(2)
df1.columns =df1.columns.map('_'.join)


df2 = df1.loc[df1['Sample_Status_'] == "Tumor"]

df3 = df2.loc[:,~df1.columns.duplicated()]
phospho_sites = df3.columns.values.tolist()

phospho_sites.remove('EGFR_Mutation_')

phospho_sites.remove('EGFR_Location_')

phospho_sites.remove('EGFR_Mutation_Status_')


phospho_sites.remove('Sample_Status_')

df3

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 174 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


,A2M_phosphoproteomics_S710,A2M_phosphoproteomics_S928,A2ML1_phosphoproteomics_S658,A2ML1_phosphoproteomics_S698,AAAS_phosphoproteomics_S462,AAAS_phosphoproteomics_S478,AAAS_phosphoproteomics_S508,AAAS_phosphoproteomics_T488S508,AAED1_phosphoproteomics_S12,AAGAB_phosphoproteomics_S106,...,ZZZ3_phosphoproteomics_S314,ZZZ3_phosphoproteomics_S381T388,ZZZ3_phosphoproteomics_S391,ZZZ3_phosphoproteomics_S396,ZZZ3_phosphoproteomics_S91,ZZZ3_phosphoproteomics_T301,EGFR_Mutation_,EGFR_Location_,EGFR_Mutation_Status_,Sample_Status_
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00977,NaN,NaN,NaN,NaN,23.810752,16.462169,NaN,NaN,19.304315,NaN,...,NaN,NaN,20.726809,NaN,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00987,NaN,NaN,NaN,NaN,24.303142,NaN,NaN,NaN,19.581772,NaN,...,NaN,10.553611,NaN,NaN,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00994,NaN,NaN,NaN,NaN,23.834852,16.514470,NaN,NaN,20.286625,NaN,...,NaN,NaN,20.389204,NaN,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00995,14.950532,NaN,NaN,NaN,24.188308,NaN,NaN,NaN,19.822521,18.761755,...,NaN,NaN,20.849815,NaN,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00997,NaN,NaN,NaN,NaN,24.178120,NaN,17.683328,NaN,19.792555,NaN,...,NaN,NaN,20.776460,14.551561,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-04277,NaN,12.966113,NaN,NaN,23.969325,16.161552,18.924554,NaN,19.760969,19.797038,...,NaN,NaN,21.123244,NaN,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3N-04278,NaN,NaN,13.672171,NaN,24.486603,16.941698,17.945032,NaN,18.729742,19.684109,...,NaN,NaN,21.169331,NaN,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3N-04279,NaN,NaN,NaN,13.404450,24.128939,16.820011,NaN,NaN,20.094552,19.822749,...,NaN,NaN,NaN,NaN,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor


In [27]:

df3.filter(like='EGFR_phosphoproteomics_Y', axis=1)

,EGFR_phosphoproteomics_Y1092,EGFR_phosphoproteomics_Y1110,EGFR_phosphoproteomics_Y1172,EGFR_phosphoproteomics_Y1197,EGFR_phosphoproteomics_Y727
Patient_ID,,,,,
C3L-00977,17.473013,NaN,20.292057,23.960771,NaN
C3L-00987,NaN,NaN,18.218061,21.464089,NaN
C3L-00994,16.886517,NaN,19.119405,22.517973,NaN
C3L-00995,NaN,NaN,NaN,22.753777,NaN
C3L-00997,NaN,16.956105,18.056164,21.358591,NaN
...,...,...,...,...,...
C3N-04277,NaN,18.134882,NaN,23.335068,NaN
C3N-04278,NaN,NaN,NaN,21.584917,NaN
C3N-04279,NaN,18.366149,19.220107,22.176533,NaN


In [28]:
df =(wrap_lin_regression(df3,"EGFR_phosphoproteomics_Y1197",comparison_columns = phospho_sites, return_all = True))
df['Cancer Type']='Hnscc'
df.to_csv("Hnscc_EGFR_Y1197_phospho.csv")

df

,Comparison,Correlation,P_value,Cancer Type
7654,EGFR_phosphoproteomics_Y1197,1.000000,0.000000e+00,Hnscc
7653,EGFR_phosphoproteomics_Y1172,0.877079,1.910986e-20,Hnscc
7645,EGFR_phosphoproteomics_S1166Y1172,0.902376,5.970836e-17,Hnscc
7652,EGFR_phosphoproteomics_Y1110,0.747328,2.047774e-14,Hnscc
7651,EGFR_phosphoproteomics_Y1092,0.916211,7.916959e-12,Hnscc
...,...,...,...,...
17957,NOP14_phosphoproteomics_S349,0.000033,9.998338e-01,Hnscc
16449,MYH14_phosphoproteomics_S1977S1991,0.000024,9.998712e-01,Hnscc
31225,ZC3H15_phosphoproteomics_S231,-0.000015,9.998833e-01,Hnscc
10504,GRASP_phosphoproteomics_S94,0.000004,9.999706e-01,Hnscc


# Lscc

In [29]:
Lscc = cptac.Lscc()

In [30]:
df1 = Lscc.join_omics_to_mutations(omics_df_name="phosphoproteomics", mutations_genes="EGFR")
df1.columns = df1.columns.droplevel(2)
df1.columns = df1.columns.droplevel(2)
df1.columns =df1.columns.map('_'.join)


df2 = df1.loc[df1['Sample_Status_'] == "Tumor"]

df3 = df2.loc[:,~df1.columns.duplicated()]
phospho_sites = df3.columns.values.tolist()

phospho_sites.remove('EGFR_Mutation_')

phospho_sites.remove('EGFR_Location_')

phospho_sites.remove('EGFR_Mutation_Status_')


phospho_sites.remove('Sample_Status_')

df3

cptac warning: In joining the somatic_mutation table, no mutations were found for the following samples, so they were filled with Wildtype_Tumor or Wildtype_Normal: 208 samples for the EGFR gene (/Users/Lindsey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py, line 1)


,A4GALT_phosphoproteomics_T324,AAAS_phosphoproteomics_S495,AAED1_phosphoproteomics_S12,AAGAB_phosphoproteomics_S310S311,AAGAB_phosphoproteomics_S311,AAK1_phosphoproteomics_S18,AAK1_phosphoproteomics_S21,AAK1_phosphoproteomics_S618T620S624,AAK1_phosphoproteomics_S624,AAK1_phosphoproteomics_S637,...,ZZZ3_phosphoproteomics_S397N422,ZZZ3_phosphoproteomics_S397T428N429,ZZZ3_phosphoproteomics_S606,ZZZ3_phosphoproteomics_S82,ZZZ3_phosphoproteomics_S89,ZZZ3_phosphoproteomics_T428N429,EGFR_Mutation_,EGFR_Location_,EGFR_Mutation_Status_,Sample_Status_
Patient_ID,,,,,,,,,,,,,,,,,,,,,
C3L-00081,NaN,1.4658,-0.3928,NaN,NaN,0.0176,-0.0391,0.5031,0.9853,-0.7181,...,NaN,-0.5079,0.6333,NaN,-1.9627,0.0360,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00415,NaN,-1.3706,-1.4682,0.8167,0.4114,1.3812,1.5283,1.5884,-2.4650,0.5195,...,0.1847,-0.1381,-0.5674,NaN,-0.5089,0.0481,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00445,NaN,0.5106,NaN,0.3627,1.9174,0.1257,-0.2341,-0.5839,NaN,-0.8685,...,NaN,NaN,-0.0649,0.2649,NaN,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00568,0.9270,0.9634,NaN,-0.6045,1.5058,0.8369,0.3980,-0.6754,1.9850,-1.3999,...,0.7660,NaN,0.5590,1.1532,-0.9744,1.5806,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3L-00603,NaN,0.9622,-0.3333,0.2873,0.5218,0.4575,0.5404,-0.0373,-0.9639,0.9122,...,NaN,-0.0831,1.0581,2.5538,0.1829,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C3N-03886,NaN,NaN,-0.5543,0.3347,-0.2520,-0.3899,-0.4523,-0.1836,-1.9700,-0.4595,...,0.6898,NaN,-0.1740,NaN,-1.1266,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3N-04124,NaN,0.0878,-0.2821,0.9885,1.1933,-0.4032,0.6692,1.4047,-0.7181,1.4531,...,NaN,NaN,-0.2623,NaN,-1.3501,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor
C3N-04127,3.6205,-0.9368,0.1505,0.0656,-0.1708,0.7750,0.3364,1.0741,1.1670,0.7325,...,NaN,NaN,0.3021,-0.7024,-0.6155,NaN,[Wildtype_Tumor],[No_mutation],Wildtype_Tumor,Tumor


In [31]:

df3.filter(like='EGFR_phosphoproteomics_Y', axis=1)



,EGFR_phosphoproteomics_Y1172,EGFR_phosphoproteomics_Y1197
Patient_ID,,
C3L-00081,1.0487,-0.7231
C3L-00415,-2.1963,-1.3811
C3L-00445,-0.2028,-1.0228
C3L-00568,0.2427,-1.4842
C3L-00603,-0.6293,-1.3100
...,...,...
C3N-03886,-2.9095,-1.6461
C3N-04124,-0.6278,-1.5549
C3N-04127,-2.0908,NaN


In [32]:
df =(wrap_lin_regression(df3,"EGFR_phosphoproteomics_Y1197",comparison_columns = phospho_sites, return_all = True))
df['Cancer Type']='Lscc'
df.to_csv("Lscc_EGFR_Y1197_phospho.csv")

df


,Comparison,Correlation,P_value,Cancer Type
10565,EGFR_phosphoproteomics_Y1197,1.000000,0.000000e+00,Lscc
10564,EGFR_phosphoproteomics_Y1172,0.912756,7.644531e-33,Lscc
10555,EGFR_phosphoproteomics_S1064,0.722468,2.977440e-16,Lscc
10554,EGFR_phosphoproteomics_S1042S1045N1053,0.732426,3.582486e-15,Lscc
10553,EGFR_phosphoproteomics_S1039,0.688452,4.739004e-15,Lscc
...,...,...,...,...
16400,ITPR2_phosphoproteomics_T1856,0.000020,9.998640e-01,Lscc
7641,CLIP2_phosphoproteomics_S202S204S207,-0.000008,9.999443e-01,Lscc
25408,PDLIM4_phosphoproteomics_S112,0.000006,9.999568e-01,Lscc
40839,WDR44_phosphoproteomics_S262T271,-0.000003,9.999820e-01,Lscc
